In [ ]:
import os
import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from mapper_functions import plot_global_tight_pcm, plot_tb_tight_pcm, plot_NA_tight_pcm


In [ ]:
start_date = datetime(2002, 1, 1)
end_date = datetime(2006, 12, 31)

start_date_str = start_date.strftime('%Y/%m/%d')
end_date_str = end_date.strftime('%Y/%m/%d')

In [ ]:

expt_name = 'LS_OLv8_M36'

# Define the path directory
root_directory = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/land_sweeper/{expt_name}/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg'

# List all tavg24 files in the root directory
files = glob.glob(os.path.join(root_directory, '*tavg24_1d_lnd_Nt*.nc'))

# Filter files based on the date range
filtered_files = []
for file in files:
    # Extract the date part from the filename
    filename = os.path.basename(file)
    date_str = filename.split('.')[-2]  # Extract the date part (e.g., 200401)
    file_date = datetime.strptime(date_str, '%Y%m')

    # Check if the file date is within the range
    if start_date <= file_date < end_date:
        filtered_files.append(file)

# Load the data 
ds_ol = xr.open_mfdataset(filtered_files, combine='nested', concat_dim="time")

# Sort the dataset by the time dimension
ds_ol = ds_ol.sortby('time')

In [ ]:
ol_tsoil1 = ds_ol['TSOIL1'].sel(time=slice(start_date_str, end_date_str))

print(ol_tsoil1.shape)

# Calculate monthly climatology
monthly_clim = ol_tsoil1.groupby('time.month').mean('time')

print("Monthly climatology shape:", monthly_clim.shape)  # Should be (12, 112573)



In [ ]:
# Open the dataset for a specific file that contains lat/lon for M36 grid
ds_latlon = xr.open_dataset('DAv7_M36.inst3_1d_lndfcstana_Nt.20150901.nc4')

# Extract longitude and latitude variables
lon = ds_latlon['lon']
lat = ds_latlon['lat']

# Determine the number of tiles based on the latitude array
n_tile = len(lat)

# Initialize an observation array with NaN values
# The array has dimensions [n_tile, 3], where:
# - Column 0 is reserved for future use
# - Column 1 stores longitude values
# - Column 2 stores latitude values
map_array = np.empty([n_tile, 3])
map_array.fill(np.nan)
map_array[:, 1] = lon
map_array[:, 2] = lat

In [ ]:
# Create month name mapping
month_names = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April',
    5: 'May', 6: 'June', 7: 'July', 8: 'August',
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

# Loop through months
for month in range(1, 13):
    # Update map array with this month's data
    map_array[:, 0] = (monthly_clim[month-1,:].values) - 273.15  # Convert from Kelvin to Celsius
    
    # Calculate min/max
    maxval = np.nanmax(map_array[:, 0])
    minval = np.nanmin(map_array[:, 0])
    
    # Create plot with month in title
    plot_global_tight_pcm(
        map_array, 
        False, 
        True, 
        f'{expt_name} {month_names[month]} Climatology:\n TSOIL_1 (Max: {maxval:.3g} Min: {minval:.3g})', 
        'ºC', 
        -20, 
        40
    )

    # Create plot with month in title
    plot_global_tight_pcm(
        map_array, 
        False, 
        True, 
        f'{expt_name} {month_names[month]} Climatology:\n TSOIL_1 (Max: {maxval:.3g} Min: {minval:.3g})', 
        'ºC', 
        0, 
        20
    )    

In [ ]:
# Loop through months
for month in range(1, 13):
    # Update map array with this month's data
    map_array[:, 0] = (monthly_clim[month-1,:].values) - 273.15  # Convert from Kelvin to Celsius
    
    # Calculate min/max
    maxval = np.nanmax(map_array[:, 0])
    minval = np.nanmin(map_array[:, 0])
    
    # Create plot with month in title
    plot_NA_tight_pcm(
        map_array, 
        False, 
        False, 
        f'{expt_name} {month_names[month]} Climatology:\n TSOIL_1 over North America', 
        'ºC', 
        -20, 
        40
    )

    # Create plot with month in title
    plot_NA_tight_pcm(
        map_array, 
        False, 
        False, 
        f'{expt_name} {month_names[month]} Climatology:\n TSOIL_1 over North America', 
        'ºC', 
        0, 
        20,
    )    